## Descarga de Imagenes Sentinel-1 a traves de la plataforma Google Earth Engine y generacion de mascaras de area construida a partir de mapas LULC-ESRI.

En este notebook se realiza la descarga de imagenes Sentinel 1 desde la plataforma Google Earth Engine (GEE). Es importante aclarar que para poder utilizar la plataforma GEE se debe estar registrado; si es necesario registrarse ingresar en https://earthengine.google.com/ y seguir los pasos alli detallados.

Ademas, se generan las mascaras de área construida a partir del mapa de Cobertura y Uso de Suelo 2021 (LULC 2021) elaborado por ESRI y descargados desde https://www.arcgis.com/apps/instant/media/index.html?appid=fc92d38533d440078f17678ebc20e8e2, a ser utilizadas en la etapa de post-procesamiento de los resultados de prediccion. 

## 1 Descarga de Imagenes Sentinel-1 

## 1.1. Importacion de librerias y definicion de variables

In [ ]:
import ee
import geemap
import os
from geemap import geojson_to_ee, ee_to_geojson

In [ ]:
#bucket con los archivos vectoriales de departamentos a ser usados como area de interes (AOI)
BUCKET_DPTO = "gs://dym-indunor-temp/immap/final/aux_data/floods_suscp/dpto_aoi/*.geojson" 

#creamos la carpeta a donde descargaremos los archivos vectoriales de los departamentos
BASE_PATH = "../"
PATH_DPTO_FILES = f"{BASE_PATH}/dpto_files/"

#se define el rango de fechas de descarga de las imagenes
startyear = 2021
endyear = 2021
month = 8
startday = 1
endday = 31

## 1.2. Autenticacion e Inicio de Google Earth Engine

Como primer paso se debe autenticar el ingreso a los servidores de GEE y dar inicio. Una vez que se corra la celda a continuacion, se le va a solicitar que ingrese a su cuenta de Google para dar acceso. Siga las instrucciones, copie el codigo de autorizacion y peguelo en la celda vacia.

In [ ]:
# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

## 1.3. Descarga de archivos vectoriales de departamentos desde bucket

In [ ]:
!mkdir -p $PATH_DPTO_FILES   #crea la carpeta
!gsutil -m cp -r $BUCKET_DPTO   $PATH_DPTO_FILES # descarga de imagenes

In [ ]:
verificar que los archivos fueron correctamente descargados
!ls $PATH_DPTO_FILES    # descomentar para ver el listado de archivos

## 1.4. Descarga de imagenes S1 por departamento

Para cada uno de los archivos de departamento, se genera una composicion de imagenes S1. Esta composicion de imagenes se filtra por:

1- tipo de instrumento (IW),

2- polarizacion (directa-VV), 

3- de orbita descendiente (que cubre toda Colombia),

4- area de interes,

5- fecha.

In [ ]:
#genero las variables de fechas
startdate = ee.Date.fromYMD(startyear,month,startday)
enddate = ee.Date.fromYMD(endyear,month,endday)

for file in os.listdir(PATH_DPTO_FILES):
    aoi = (geojson_to_ee(f'{PATH_DPTO_FILES}{file}')).geometry() #el archivo geojson se convierte a formato compatible con GEE
    
    # se establece una funcion de recorte de la imagen segun el area de interes
    def corte(image):
        return image.clip(roi) 
    
    #coleccion de imagenes de Sentinel 1
    sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')

    # polarizacion directa
    VVd = sentinel1 \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .map(corte) \
        .select(['VV']) \
        .filter(ee.Filter.eq('resolution_meters',10)) \
        .filterDate(startdate, enddate).median()
    
    #polarizacion cruzada
    VHd = sentinel1 \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .map(corte) \
        .select(['VH']) \
        .filter(ee.Filter.eq('resolution_meters',10)) \
        .filterDate(startdate, enddate).median()
    
    # suavizado para reduccion de 'specle noise'
    smoothing_radius = 10
    VHd_filtered = VHd.focal_mean(smoothing_radius, 'circle', 'meters')
    VVd_filtered = VVd.focal_mean(smoothing_radius, 'circle', 'meters')
    
    # 3ra componente en funcion de polarizaciones directa y cruzada
    VHVVd_filtered = VHd_filtered.divide(VVd_filtered)
    
    # imagen final
    image_desc_2021 = VHd_filtered.addBands(VVd_filtered).addBands(VHVVd_filtered)
    
    # descarga de imagen S1 por departamento a bucket
    task = ee.batch.Export.image.CloudStorage(
                    image= image_desc_2021,
                    bucket = 'dym-indunor-temp/',
                    fileNamePrefix = f"immap/final/aux_data/inf_settl/s1_images/s1_",
                    maxPixels = 8030040147504,
                    scale=10,
                    region= aoi,  
                    crs='EPSG:4326')                                 
    
    task.start()

-------------------------------------------------------------------------------

## 2. Generacion de mascaras de area construida

## 2.1. Importacion de librerias y definicion de variables

In [ ]:
from osgeo import gdal
from pathlib import Path
import glob
import numpy as np
from os import remove
import subprocess
from os import remove
import geopandas as gpd

In [ ]:
#bucket con los archivos a utilizar
#LULC
BUCKET_LULC = "gs://dym-indunor-temp/immap/final/aux_data/inf_settl/LULC/*.tif"
#MUNICIPIOS
BUCKET_MUN = "gs://dym-indunor-temp/immap/v3_1/data/munis_v3.gpkg"

#creamos las carpetas a donde descargaremos los archivos desde el bucket
BASE_PATH = "../"
PATH_LULC_FILES = f"{BASE_PATH}/LULC_files/"
PATH_MUN_FILES = f"{BASE_PATH}/MUN_files/"

#carpeta de guardado para cada achivo vectorial de municipio
#PATH_MUN_INDIVIDUALES = f'{BASE_PATH}/MUNICIPIOS_INDIVIDUALES/'

#carpeta de guardado de las mascara de area construida
PATH_LULC_MASK = f"{BASE_PATH}/LULC_MASK/"

#PARAMETROS GENERALES
aoi = 'colombia'
prj = "32618"

## 2.2. Descarga de archivos desde bucket

In [ ]:
#LULC
!mkdir -p $PATH_LULC_FILES   #crea la carpeta
!gsutil -m cp -r $BUCKET_LULC   $PATH_LULC_FILES # descarga de archivos

#MUNICIPIOS
!mkdir -p $PATH_MUN_FILES   #crea la carpeta
!gsutil -m cp -r $BUCKET_MUN   $PATH_MUN_FILES # descarga de archivos

## 2.3. Generar mosaico LULC

In [ ]:
# unificar la proyeccion de los archivos LULC
for file in os.listdir(PATH_LULC_FILES):
    if file.endswith('.tif'):
        ds = gdal.Open(f'{PATH_LULC_FILES}{file}')
        output_file = f'{PATH_LULC_FILES}{file[0:-4]}_epsg{prj}.vrt'
        dsReprj = gdal.Warp(output_file, ds, dstSRS=f'EPSG:{prj}')
        ds = None
        dsReprj = None

In [ ]:
# mosaico
lista = []
for file in os.listdir(f'{PATH_LULC_FILES}'):
    if file.endswith(".vrt"):
        lista.append(file)
        
file_txt = 'files.txt'

textfile = open(f'{PATH_LULC_FILES}/{file_txt}', "w")
for element in lista:
    textfile.write(element + "\n")
textfile.close()

os.chdir(PATH_LULC_FILES)
cmd = f'gdalbuildvrt lulc2021_mosaic.vrt -input_file_list {file_txt}'
subprocess.run(cmd, shell=True)

In [ ]:
for file in os.listdir(PATH_MUN_FILES):
    mun = gpd.read_file(Path(f'{PATH_MUN_FILES}{file}'))
    for i, row in mun.iterrows():
        municipio = row['COD_MUNICIPIO']
        dpto = row['DPTO_COD']
        outpath_folder = f'{PATH_MUN_FILES}{dpto}/{municipio}/'
        os.makedirs(outpath_folder, exist_ok=True)
        outpath_municipio = f'{outpath_folder}{municipio}.geojson'
        gpd.GeoDataFrame(geometry=list(row.geometry)).to_file(outpath_municipio)
        
        mask_folder = f'{PATH_LULC_FILES}BUILT_MASK/{dpto}/{municipio}/'
        os.makedirs(mask_folder, exist_ok=True)
        
        for file in os.listdir(outpath_folder):
            search_criteria_geojson = f'{outpath_folder}*.geojson'
            subset_files = glob.glob(search_criteria_geojson)
            
            for fl in subset_files:
                infile = f'{PATH_LULC_FILES}lulc2021_mosaic.vrt'
                outfile = f'{mask_folder}{municipio}.vrt'
                ds = gdal.Open(infile)
                ds_recorte = gdal.Warp(outfile, ds, cutlineDSName = fl, cropToCutline = True, dstNodata = np.nan)
                ds_recorte = None
                
                # set no data as nan and mask out non urban classes
                raster = gdal.Open(outfile)
                noDataVal = raster.GetRasterBand(1).GetNoDataValue() 
                array = raster.GetRasterBand(1).ReadAsArray(0,0,raster.RasterXSize,raster.RasterYSize).astype(float) 
                reclass = array
                reclass[np.where(array == noDataVal)] = np.nan
                reclass[np.where(array != 7)] = 0
                reclass[np.where(array == 7)] = 1
                
                 #save new file
                driver = gdal.GetDriverByName('GTiff')
                file = driver.Create(f'{mask_folder}/{municipio}.tif', raster.RasterXSize,raster.RasterYSize, 1, gdal.GDT_Float32)
                file.GetRasterBand(1).WriteArray(reclass)
                file.GetRasterBand(1).SetNoDataValue(np.nan)

                # spatial ref system
                file.SetProjection(raster.GetProjection())
                file.SetGeoTransform(raster.GetGeoTransform())
                file.FlushCache()